In [253]:
# Dependencies
import pandas as pd
from pathlib import Path
from pprint import pprint


In [254]:
# reading the CSV and creating a dataframe from the information
original_crime_file = Path("offensesperori_final.csv")
og_crime_df = pd.read_csv(original_crime_file)

In [255]:
# use the starts with method to clean the dataframe to only contain California agencies
california_agencies = og_crime_df[og_crime_df['ori_number'].str.startswith('CA')]

In [256]:
# reads in files that contain agency names
ca_agency_names = Path("orinum_agencyname.csv")
ca_names = pd.read_csv(ca_agency_names)

#change the name of the ORI column to match our original dataframe
ca_names = ca_names.rename(columns={'ORI':'ori_number'})

In [257]:
# completes a right merge to add the agency names to the california
merge_1 = pd.merge(california_agencies, ca_names, how = 'right', on='ori_number')

In [258]:
#reads in new csv that contains county codes and county names
ca_counties = Path("state_agencies.csv")
county_names = pd.read_csv(ca_counties)

#filters for only the California information as the data frame includes New York and Illinois as well 
ca_county_names = county_names.loc[county_names["state_abbr"] == "CA"]

#delete columns that don't need to be put into merged dataframe
del ca_county_names['Unnamed: 0']
del ca_county_names['state_abbr']
del ca_county_names['agency_name']

#changes the name of the column being merged on to match existing df
ca_county_names = ca_county_names.rename(columns={'ori':'ori_number'})

In [259]:
#merge ca_county_names df into the california dataframe to now add the county-level information
#Using an outer merge to capture all of the information in both dataframes
california_crime_counties = pd.merge(merge_1, ca_county_names, how='outer', on='ori_number')

In [65]:
#dropping all rows with NaN values
california_crime_counties.dropna(inplace=True)


In [260]:
# Convert 'data_year' and 'agency_id' columns to integers
california_crime_counties['data_year'] = california_crime_counties['data_year'].astype(int)
california_crime_counties['agency_id'] = california_crime_counties['agency_id'].astype(int)

# Convert integers to strings
california_crime_counties['data_year'] = california_crime_counties['data_year'].astype(str)
california_crime_counties['agency_id'] = california_crime_counties['agency_id'].astype(str)

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [261]:
# set the new types for each column
new_types = {'Aggravated Assault': int,
             'All Other Offenses (Except Traffic)': int,
             'Arson': int,
             'Burglary': int,
             'Curfew and Loitering Law Violations': int,
             'Disorderly Conduct': int,
             'Driving Under the Influence': int,
             'Drug Abuse Violations - Grand Total': int,
             'Drunkenness': int,
             'Embezzlement': int,
             'Forgery and Counterfeiting': int,
             'Fraud': int,
             'Gambling - Total': int,
             'Human Trafficking - Commercial Sex Acts': int,
             'Human Trafficking - Involuntary Servitude': int,
             'Larceny - Theft': int,
             'Liquor Laws': int,
             'Manslaughter by Negligence': int,
             'Motor Vehicle Theft': int,
             'Murder and Nonnegligent Manslaughter': int,
             'Offenses Against the Family and Children': int,
             'Prostitution and Commercialized Vice': int,
             'Rape': int,
             'Robbery': int,
             'Simple Assault': int,
             'Stolen Property: Buying, Receiving, Possessing': int,
             'Suspicion': int,
             'Vagrancy': int,
             'Vandalism': int,
             'Weapons: Carrying, Possessing, Etc.': int,
             'Sex Offenses (Except Rape, and Prostitution and Commercialized Vice)': int
          }
#using the dictionaries to set new types
california_crime_counties = california_crime_counties.astype(new_types)
#california_crime_counties['data_year'] = california_crime_counties['data_year'].astype(str)
california_crime_counties

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer: Error while type casting for column 'Aggravated Assault'

In [128]:
#county level with all data

#aggregate by crime types
county_full =  california_crime_counties.groupby(['county_name','data_year']).agg({
            'Aggravated Assault': 'sum',
             'All Other Offenses (Except Traffic)': 'sum',
             'Arson': 'sum',
             'Burglary': 'sum',
             'Curfew and Loitering Law Violations': 'sum',
             'Disorderly Conduct': 'sum',
             'Driving Under the Influence': 'sum',
             'Drug Abuse Violations - Grand Total': 'sum',
             'Drunkenness': 'sum',
             'Embezzlement': 'sum',
             'Forgery and Counterfeiting': 'sum',
             'Fraud': 'sum',
             'Gambling - Total': 'sum',
             'Human Trafficking - Commercial Sex Acts': 'sum',
             'Human Trafficking - Involuntary Servitude': 'sum',
             'Larceny - Theft': 'sum',
             'Liquor Laws': 'sum',
             'Manslaughter by Negligence': 'sum',
             'Motor Vehicle Theft': 'sum',
             'Murder and Nonnegligent Manslaughter': 'sum',
             'Offenses Against the Family and Children': 'sum',
             'Prostitution and Commercialized Vice': 'sum',
             'Rape': 'sum',
             'Robbery': 'sum',
             'Simple Assault': 'sum',
             'Stolen Property: Buying, Receiving, Possessing': 'sum',
             'Suspicion': 'sum',
             'Vagrancy': 'sum',
             'Vandalism': 'sum',
             'Weapons: Carrying, Possessing, Etc.': 'sum',
             'Sex Offenses (Except Rape, and Prostitution and Commercialized Vice)': 'sum'
})

# Calculate the count of unique agencies for each county and year
unique_agency_count_1 = california_crime_counties.groupby(['county_name', 'data_year'])['AGENCY_NAME'].nunique().reset_index()

# # Merge the unique agency count with county_level_data
county_full = county_full.merge(unique_agency_count_1, on=['county_name', 'data_year'], how='left')
county_full.rename(columns={'AGENCY_NAME': 'agency_count'}, inplace=True)

#create a csv file from the dataframe
county_full.to_csv('Output/county_level_including_all_agencies_and_available_years.csv')

In [130]:
#create a dataframe at the agency level with all years and all agencies
agency_level_full = california_crime_counties.groupby(['AGENCY_NAME','data_year']).agg({
    'Aggravated Assault': 'sum',
             'All Other Offenses (Except Traffic)': 'sum',
             'Arson': 'sum',
             'Burglary': 'sum',
             'Curfew and Loitering Law Violations': 'sum',
             'Disorderly Conduct': 'sum',
             'Driving Under the Influence': 'sum',
             'Drug Abuse Violations - Grand Total': 'sum',
             'Drunkenness': 'sum',
             'Embezzlement': 'sum',
             'Forgery and Counterfeiting': 'sum',
             'Fraud': 'sum',
             'Gambling - Total': 'sum',
             'Human Trafficking - Commercial Sex Acts': 'sum',
             'Human Trafficking - Involuntary Servitude': 'sum',
             'Larceny - Theft': 'sum',
             'Liquor Laws': 'sum',
             'Manslaughter by Negligence': 'sum',
             'Motor Vehicle Theft': 'sum',
             'Murder and Nonnegligent Manslaughter': 'sum',
             'Offenses Against the Family and Children': 'sum',
             'Prostitution and Commercialized Vice': 'sum',
             'Rape': 'sum',
             'Robbery': 'sum',
             'Simple Assault': 'sum',
             'Stolen Property: Buying, Receiving, Possessing': 'sum',
             'Suspicion': 'sum',
             'Vagrancy': 'sum',
             'Vandalism': 'sum',
             'Weapons: Carrying, Possessing, Etc.': 'sum',
             'Sex Offenses (Except Rape, and Prostitution and Commercialized Vice)': 'sum'
})

#create a csv file from the dataframe
agency_level_full.to_csv('Output/agency_level_including_all_agencies_and_available_years.csv')

In [149]:
#create a dataframe with the consolidated crime categories
#copy the dataframe
consolidated_crime_full_counties = california_crime_counties.copy()

#these are the crimes that should be mapped to the crimes against persons category
consolidated_crime_full_counties["Crimes Against Persons"] = consolidated_crime_full_counties["Aggravated Assault"] + consolidated_crime_full_counties["Human Trafficking - Commercial Sex Acts"] + consolidated_crime_full_counties["Human Trafficking - Involuntary Servitude"] + consolidated_crime_full_counties["Manslaughter by Negligence"] + consolidated_crime_full_counties["Murder and Nonnegligent Manslaughter"] + consolidated_crime_full_counties["Rape"] + consolidated_crime_full_counties["Simple Assault"] + consolidated_crime_full_counties["Sex Offenses (Except Rape, and Prostitution and Commercialized Vice)"]

#these are the crimes that should be mapped to the crimes against society category
consolidated_crime_full_counties["Crimes Against Society"] = consolidated_crime_full_counties["Curfew and Loitering Law Violations"] + consolidated_crime_full_counties["Disorderly Conduct"] + consolidated_crime_full_counties["Driving Under the Influence"] + consolidated_crime_full_counties["Drug Abuse Violations - Grand Total"] + consolidated_crime_full_counties["Drunkenness"] + consolidated_crime_full_counties["Gambling - Total"] + consolidated_crime_full_counties["Liquor Laws"] + consolidated_crime_full_counties["Offenses Against the Family and Children"] + consolidated_crime_full_counties["Prostitution and Commercialized Vice"]+ consolidated_crime_full_counties["Vagrancy"]

#these are the crimes that should be mapped to the crimes against property category
consolidated_crime_full_counties["Crimes Against Property"] = consolidated_crime_full_counties["Arson"] + consolidated_crime_full_counties["Burglary"] + consolidated_crime_full_counties["Embezzlement"] + consolidated_crime_full_counties["Forgery and Counterfeiting"] + consolidated_crime_full_counties["Fraud"] + consolidated_crime_full_counties["Larceny - Theft"] + consolidated_crime_full_counties["Motor Vehicle Theft"] + consolidated_crime_full_counties["Robbery"]+ consolidated_crime_full_counties["Stolen Property: Buying, Receiving, Possessing"] + consolidated_crime_full_counties["Vandalism"]+ consolidated_crime_full_counties["Weapons: Carrying, Possessing, Etc."]


#create a list of columns to remove
columns_to_drop_1 = [
    'Unnamed: 0','Aggravated Assault', 'Arson', 'Burglary',
       'Curfew and Loitering Law Violations', 'Disorderly Conduct',
       'Driving Under the Influence', 'Drug Abuse Violations - Grand Total',
       'Drunkenness', 'Embezzlement', 'Forgery and Counterfeiting', 'Fraud',
       'Gambling - Total', 'Human Trafficking - Commercial Sex Acts',
       'Human Trafficking - Involuntary Servitude', 'Larceny - Theft',
       'Liquor Laws', 'Manslaughter by Negligence', 'Motor Vehicle Theft',
       'Murder and Nonnegligent Manslaughter',
       'Offenses Against the Family and Children',
       'Prostitution and Commercialized Vice', 'Rape', 'Robbery',
       'Simple Assault', 'Stolen Property: Buying, Receiving, Possessing', 'Vagrancy', 'Vandalism',
       'Weapons: Carrying, Possessing, Etc.',
       'Sex Offenses (Except Rape, and Prostitution and Commercialized Vice)' 
]

#deletes columns we no longer need
consolidated_crime_full_counties.drop(columns=columns_to_drop_1, inplace=True)

#reorder the columns in the dataframe
new_order_1 = ["data_year",
             "county_name",
             "AGENCY_NAME",
             "agency_id",
             "Crimes Against Persons",
             "Crimes Against Society",
             "Crimes Against Property",
             "All Other Offenses (Except Traffic)",
             "Suspicion",
             "latitude",
             "longitude",
             "ori_number",
             "agency_type_name",
            ]
#changes the order of the dataframe
consolidated_crime_full_counties = consolidated_crime_full_counties.reindex(columns=new_order_1)
consolidated_crime_full_counties_2 = consolidated_crime_full_counties

#organizes the dataframe to be grouped by county
consolidated_crime_full_counties = consolidated_crime_full_counties.groupby(['county_name','data_year']).agg({
    'Crimes Against Persons': 'sum',
             'Crimes Against Society': 'sum',
             'Crimes Against Property': 'sum',
})

#creates a csv
consolidated_crime_full_counties.to_csv('Output/county_level_consolidated_all_agencies_all_available_years.csv')

In [148]:
#creates a dataframe of the consolidated crime categories at the agency level
#organizes the dataframe to be grouped by county

consolidated_crime_full_agencies = consolidated_crime_full_counties_2.groupby(['AGENCY_NAME','data_year']).agg({
    'Crimes Against Persons': 'sum',
             'Crimes Against Society': 'sum',
             'Crimes Against Property': 'sum',
})

consolidated_crime_full_agencies.to_csv('Output/agency_level_consolidated_all_agencies_all_available_years.csv')

In [97]:
#this snippet filters the data for agencies that have data for all six years
#to keep this consistent, it makes the most sense to only include agencies that have data for all years
#otherwise, our findings would misrepresent the realities for years with missing information

all_years_included = california_crime_counties.loc[california_crime_counties["Unnamed: 0"] == 5.0]


,Unnamed: 0,data_year,Aggravated Assault,All Other Offenses (Except Traffic),Arson,Burglary,Curfew and Loitering Law Violations,Disorderly Conduct,Driving Under the Influence,Drug Abuse Violations - Grand Total,...,Vandalism,"Weapons: Carrying, Possessing, Etc.","Sex Offenses (Except Rape, and Prostitution and Commercialized Vice)",ori_number,AGENCY_NAME,agency_id,agency_type_name,county_name,latitude,longitude
10,5.0,2022,47,151,8,14,0,71,115,80,...,38,30,7,CA0010100,Alameda Police Department,938,City,ALAMEDA,37.766888,-122.242730
18,5.0,2022,10,37,4,5,0,6,39,7,...,5,6,1,CA0010200,Albany Police Department,939,City,ALAMEDA,37.887703,-122.298250
29,5.0,2022,24,33,1,18,0,11,18,29,...,7,25,0,CA0010400,Emeryville Police Department,941,City,ALAMEDA,37.648134,-121.913291
50,5.0,2022,26,168,0,17,0,28,62,48,...,22,16,1,CA0010800,Newark Police Department,945,City,ALAMEDA,37.536236,-122.029510
146,5.0,2022,61,308,8,10,0,276,40,23,...,39,18,0,CA0040400,Oroville Police Department,979,City,BUTTE,39.508167,-121.554825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2319,5.0,2022,95,213,5,23,0,24,245,139,...,34,50,2,CA0540600,Tulare Police Department,1773,City,TULARE,36.207127,-119.343150
2369,5.0,2022,57,96,3,8,0,6,35,97,...,22,37,0,CA0560600,Santa Paula Police Department,1791,City,VENTURA,34.356910,-119.061360
2395,5.0,2022,39,257,1,13,0,70,81,55,...,23,17,2,CA0570100,Davis Police Department,1807,City,YOLO,38.679595,-121.902444
2408,5.0,2022,71,771,12,15,0,95,182,144,...,50,39,1,CA0570300,Woodland Police Department,1809,City,YOLO,38.679595,-121.902444


In [70]:
#isolate the agencies with data for all years
agencies_all_years = all_years_included["AGENCY_NAME"].values

array(['Alameda Police Department', 'Albany Police Department',
       'Emeryville Police Department', 'Newark Police Department',
       'Oroville Police Department',
       "Calaveras County Sheriff's Department",
       'Williams Police Department', 'Antioch Police Department',
       'Clayton Police Department', 'Concord Police Department',
       'El Cerrito Police Department', 'Richmond Police Department',
       'Moraga Police Department',
       "El Dorado County Sheriff's Department",
       'Placerville Police Department', 'Fresno Police Department',
       "Glenn County Sheriff's Department", 'Ferndale Police Department',
       'Fortuna Police Department', 'Rio Dell Police Department',
       'Delano Police Department', 'Ridgecrest Police Department',
       "Kings County Sheriff's Department", 'Corcoran Police Department',
       'Hanford Police Department', 'Lemoore Police Department',
       "Lake County Sheriff's Department", 'Lakeport Police Department',
       "Lassen

In [101]:
#creates a dataframe with only agencies that have the full 6 years of data
agencies_with_all_years = california_crime_counties[california_crime_counties["AGENCY_NAME"].isin(agencies_all_years)]

,Unnamed: 0,data_year,Aggravated Assault,All Other Offenses (Except Traffic),Arson,Burglary,Curfew and Loitering Law Violations,Disorderly Conduct,Driving Under the Influence,Drug Abuse Violations - Grand Total,...,Vandalism,"Weapons: Carrying, Possessing, Etc.","Sex Offenses (Except Rape, and Prostitution and Commercialized Vice)",ori_number,AGENCY_NAME,agency_id,agency_type_name,county_name,latitude,longitude
5,0.0,2017,97,273,4,29,0,2,114,110,...,6,18,13,CA0010100,Alameda Police Department,938,City,ALAMEDA,37.766888,-122.242730
6,1.0,2018,109,250,0,45,0,5,87,200,...,14,28,12,CA0010100,Alameda Police Department,938,City,ALAMEDA,37.766888,-122.242730
7,2.0,2019,90,293,3,32,0,4,121,317,...,17,31,12,CA0010100,Alameda Police Department,938,City,ALAMEDA,37.766888,-122.242730
8,3.0,2020,87,182,5,22,0,3,69,144,...,22,12,17,CA0010100,Alameda Police Department,938,City,ALAMEDA,37.766888,-122.242730
9,4.0,2021,2,28,0,1,0,16,32,7,...,1,3,3,CA0010100,Alameda Police Department,938,City,ALAMEDA,37.766888,-122.242730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2415,1.0,2018,4,20,0,0,0,6,6,8,...,0,3,0,CA0579700,UC Davis Police Department,1813,University or College,YOLO,38.679595,-121.902444
2416,2.0,2019,0,16,1,10,0,10,8,10,...,3,2,1,CA0579700,UC Davis Police Department,1813,University or College,YOLO,38.679595,-121.902444
2417,3.0,2020,0,8,0,2,0,3,1,9,...,0,1,1,CA0579700,UC Davis Police Department,1813,University or College,YOLO,38.679595,-121.902444
2418,4.0,2021,1,3,0,0,0,0,0,1,...,0,0,0,CA0579700,UC Davis Police Department,1813,University or College,YOLO,38.679595,-121.902444


In [104]:
#This dataframe aggregates information at the county level for every year in the data

county_level_data = agencies_with_all_years.groupby(['county_name','data_year']).agg({
    'Aggravated Assault': 'sum',
             'All Other Offenses (Except Traffic)': 'sum',
             'Arson': 'sum',
             'Burglary': 'sum',
             'Curfew and Loitering Law Violations': 'sum',
             'Disorderly Conduct': 'sum',
             'Driving Under the Influence': 'sum',
             'Drug Abuse Violations - Grand Total': 'sum',
             'Drunkenness': 'sum',
             'Embezzlement': 'sum',
             'Forgery and Counterfeiting': 'sum',
             'Fraud': 'sum',
             'Gambling - Total': 'sum',
             'Human Trafficking - Commercial Sex Acts': 'sum',
             'Human Trafficking - Involuntary Servitude': 'sum',
             'Larceny - Theft': 'sum',
             'Liquor Laws': 'sum',
             'Manslaughter by Negligence': 'sum',
             'Motor Vehicle Theft': 'sum',
             'Murder and Nonnegligent Manslaughter': 'sum',
             'Offenses Against the Family and Children': 'sum',
             'Prostitution and Commercialized Vice': 'sum',
             'Rape': 'sum',
             'Robbery': 'sum',
             'Simple Assault': 'sum',
             'Stolen Property: Buying, Receiving, Possessing': 'sum',
             'Suspicion': 'sum',
             'Vagrancy': 'sum',
             'Vandalism': 'sum',
             'Weapons: Carrying, Possessing, Etc.': 'sum',
             'Sex Offenses (Except Rape, and Prostitution and Commercialized Vice)': 'sum'
})


Aggravated Assault  \
county_name data_year                       
ALAMEDA     2017                      196   
            2018                      196   
            2019                      190   
            2020                      173   
            2021                       21   
...                                   ...   
YOLO        2018                      181   
            2019                      195   
            2020                      173   
            2021                       40   
            2022                      110   

                       All Other Offenses (Except Traffic)  Arson  Burglary  \
county_name data_year                                                         
ALAMEDA     2017                                       666      6       105   
            2018                                       568      3       103   
            2019                                       622      8        83   
            2020                                       390      6        68   
            2021                                       136      1        12   
...                                                    ...    ...       ...   
YOLO        2018                                       848     13       105   
            2019                                       792     14       113   
            2020                                       534     13        89   
            2021                                       233      2         7   
            2022                                      1044     13        32   

                       Curfew and Loitering Law Violations  \
county_name data_year                                        
ALAMEDA     2017                                         0   
            2018                                         0   
            2019                                         0   
            2020                                         0   
            2021                                         0   
...                                                    ...   
YOLO        2018                                         1   
            2019                                         1   
            2020                                         0   
            2021                                         0   
            2022                                         0   

                       Disorderly Conduct  Driving Under the Influence  \
county_name data_year                                                    
ALAMEDA     2017                        5                          302   
            2018                       15                          308   
            2019                       12                          337   
            2020                        4                          193   
            2021                       33                          100   
...                                   ...                          ...   
YOLO        2018                       11                          182   
            2019                       13                          169   
            2020                        7                          122   
            2021                       44                           61   
            2022                      170                          274   

                       Drug Abuse Violations - Grand Total  Drunkenness  \
county_name data_year                                                     
ALAMEDA     2017                                       514          301   
            2018                                       606          214   
            2019                                       649           59   
            2020                                       402           26   
            2021                                        48            0   
...                                                    ...          ...   
YOLO        2018                       

In [114]:
# Calculate the count of unique agencies for each county and year
unique_agency_count = agencies_with_all_years.groupby(['county_name', 'data_year'])['AGENCY_NAME'].nunique().reset_index()

# Merge the unique agency count with county_level_data
county_level_data = county_level_data.merge(unique_agency_count, on=['county_name', 'data_year'], how='left')
county_level_data.rename(columns={'AGENCY_NAME': 'agency_count'}, inplace=True)


In [115]:
#create a csv for the county level dataframe

county_level_data.to_csv('Output/county_level_crime_data.csv')

In [73]:
#agency level
agency_level_data = agencies_with_all_years.groupby(['AGENCY_NAME','data_year']).agg({
    'Aggravated Assault': 'sum',
             'All Other Offenses (Except Traffic)': 'sum',
             'Arson': 'sum',
             'Burglary': 'sum',
             'Curfew and Loitering Law Violations': 'sum',
             'Disorderly Conduct': 'sum',
             'Driving Under the Influence': 'sum',
             'Drug Abuse Violations - Grand Total': 'sum',
             'Drunkenness': 'sum',
             'Embezzlement': 'sum',
             'Forgery and Counterfeiting': 'sum',
             'Fraud': 'sum',
             'Gambling - Total': 'sum',
             'Human Trafficking - Commercial Sex Acts': 'sum',
             'Human Trafficking - Involuntary Servitude': 'sum',
             'Larceny - Theft': 'sum',
             'Liquor Laws': 'sum',
             'Manslaughter by Negligence': 'sum',
             'Motor Vehicle Theft': 'sum',
             'Murder and Nonnegligent Manslaughter': 'sum',
             'Offenses Against the Family and Children': 'sum',
             'Prostitution and Commercialized Vice': 'sum',
             'Rape': 'sum',
             'Robbery': 'sum',
             'Simple Assault': 'sum',
             'Stolen Property: Buying, Receiving, Possessing': 'sum',
             'Suspicion': 'sum',
             'Vagrancy': 'sum',
             'Vandalism': 'sum',
             'Weapons: Carrying, Possessing, Etc.': 'sum',
             'Sex Offenses (Except Rape, and Prostitution and Commercialized Vice)': 'sum'
})


Aggravated Assault  \
AGENCY_NAME                 data_year                       
Alameda Police Department   2017                       97   
                            2018                      109   
                            2019                       90   
                            2020                       87   
                            2021                        2   
...                                                   ...   
Yuba City Police Department 2018                      245   
                            2019                      250   
                            2020                      228   
                            2021                        7   
                            2022                      125   

                                       All Other Offenses (Except Traffic)  \
AGENCY_NAME                 data_year                                        
Alameda Police Department   2017                                       273   
                            2018                                       250   
                            2019                                       293   
                            2020                                       182   
                            2021                                        28   
...                                                                    ...   
Yuba City Police Department 2018                                       959   
                            2019                                       943   
                            2020                                       623   
                            2021                                        87   
                            2022                                       860   

                                       Arson  Burglary  \
AGENCY_NAME                 data_year                    
Alameda Police Department   2017           4        29   
                            2018           0        45   
                            2019           3        32   
                            2020           5        22   
                            2021           0         1   
...                                      ...       ...   
Yuba City Police Department 2018           3        81   
                            2019           6       102   
                            2020           4        83   
                            2021           0         6   
                            2022           2        34   

                                       Curfew and Loitering Law Violations  \
AGENCY_NAME                 data_year                                        
Alameda Police Department   2017                                         0   
                            2018                                         0   
                            2019                                         0   
                            2020                                         0   
                            2021                                         0   
...                                                                    ...   
Yuba City Police Department 2018                                         4   
                            2019                                         0   
                            2020                                         6   
                            2021                                         0   
                            2022                                         0   

                                       Disorderly Conduct  \
AGENCY_NAME                 data_year                       
Alameda Police Department   2017                        2   
                            2018                        5   
                            2019                        4   
                            2020                        3   
                            2021                       16   
...                              

In [76]:
#create a csv for the agency level dataframe

agency_level_data.to_csv('Output/agency_level_crime_data.csv')

In [110]:
#putting these categories into new columns in the dataframe

#copy the dataframe
consolidated_crime_df = agencies_with_all_years.copy()

#these are the crimes that should be mapped to the crimes against persons category
consolidated_crime_df["Crimes Against Persons"] = consolidated_crime_df["Aggravated Assault"] + consolidated_crime_df["Human Trafficking - Commercial Sex Acts"] + consolidated_crime_df["Human Trafficking - Involuntary Servitude"] + consolidated_crime_df["Manslaughter by Negligence"] + consolidated_crime_df["Murder and Nonnegligent Manslaughter"] + consolidated_crime_df["Rape"] + consolidated_crime_df["Simple Assault"] + consolidated_crime_df["Sex Offenses (Except Rape, and Prostitution and Commercialized Vice)"]

#these are the crimes that should be mapped to the crimes against society category
consolidated_crime_df["Crimes Against Society"] = consolidated_crime_df["Curfew and Loitering Law Violations"] + consolidated_crime_df["Disorderly Conduct"] + consolidated_crime_df["Driving Under the Influence"] + consolidated_crime_df["Drug Abuse Violations - Grand Total"] + consolidated_crime_df["Drunkenness"] + consolidated_crime_df["Gambling - Total"] + consolidated_crime_df["Liquor Laws"] + consolidated_crime_df["Offenses Against the Family and Children"] + consolidated_crime_df["Prostitution and Commercialized Vice"]+ consolidated_crime_df["Vagrancy"]

#these are the crimes that should be mapped to the crimes against property category
consolidated_crime_df["Crimes Against Property"] = consolidated_crime_df["Arson"] + consolidated_crime_df["Burglary"] + consolidated_crime_df["Embezzlement"] + consolidated_crime_df["Forgery and Counterfeiting"] + consolidated_crime_df["Fraud"] + consolidated_crime_df["Larceny - Theft"] + consolidated_crime_df["Motor Vehicle Theft"] + consolidated_crime_df["Robbery"]+ consolidated_crime_df["Stolen Property: Buying, Receiving, Possessing"] + consolidated_crime_df["Vandalism"]+ consolidated_crime_df["Weapons: Carrying, Possessing, Etc."]

#del consolidated_crime_df["Crimes Against Property"]

#create a list of columns to remove
columns_to_drop = [
    'Unnamed: 0','Aggravated Assault', 'Arson', 'Burglary',
       'Curfew and Loitering Law Violations', 'Disorderly Conduct',
       'Driving Under the Influence', 'Drug Abuse Violations - Grand Total',
       'Drunkenness', 'Embezzlement', 'Forgery and Counterfeiting', 'Fraud',
       'Gambling - Total', 'Human Trafficking - Commercial Sex Acts',
       'Human Trafficking - Involuntary Servitude', 'Larceny - Theft',
       'Liquor Laws', 'Manslaughter by Negligence', 'Motor Vehicle Theft',
       'Murder and Nonnegligent Manslaughter',
       'Offenses Against the Family and Children',
       'Prostitution and Commercialized Vice', 'Rape', 'Robbery',
       'Simple Assault', 'Stolen Property: Buying, Receiving, Possessing', 'Vagrancy', 'Vandalism',
       'Weapons: Carrying, Possessing, Etc.',
       'Sex Offenses (Except Rape, and Prostitution and Commercialized Vice)' 
]
#deletes columns we no longer need
consolidated_crime_df.drop(columns=columns_to_drop, inplace=True)


,data_year,All Other Offenses (Except Traffic),Suspicion,ori_number,AGENCY_NAME,agency_id,agency_type_name,county_name,latitude,longitude,Crimes Against Persons,Crimes Against Society,Crimes Against Property
5,2017,273,0,CA0010100,Alameda Police Department,938,City,ALAMEDA,37.766888,-122.242730,236,457,231
6,2018,250,0,CA0010100,Alameda Police Department,938,City,ALAMEDA,37.766888,-122.242730,257,445,211
7,2019,293,0,CA0010100,Alameda Police Department,938,City,ALAMEDA,37.766888,-122.242730,300,445,223
8,2020,182,0,CA0010100,Alameda Police Department,938,City,ALAMEDA,37.766888,-122.242730,266,217,178
9,2021,28,0,CA0010100,Alameda Police Department,938,City,ALAMEDA,37.766888,-122.242730,27,55,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2415,2018,20,0,CA0579700,UC Davis Police Department,1813,University or College,YOLO,38.679595,-121.902444,17,27,13
2416,2019,16,0,CA0579700,UC Davis Police Department,1813,University or College,YOLO,38.679595,-121.902444,9,34,25
2417,2020,8,0,CA0579700,UC Davis Police Department,1813,University or College,YOLO,38.679595,-121.902444,8,13,7
2418,2021,3,0,CA0579700,UC Davis Police Department,1813,University or College,YOLO,38.679595,-121.902444,1,1,0


In [111]:
#creates a list of new column order
new_order = ["data_year",
             "county_name",
             "AGENCY_NAME",
             "agency_id",
             "Crimes Against Persons",
             "Crimes Against Society",
             "Crimes Against Property",
             "All Other Offenses (Except Traffic)",
             "Suspicion",
             "latitude",
             "longitude",
             "ori_number",
             "agency_type_name",
            ]
#changes the order of the dataframe
consolidated_crime_df = consolidated_crime_df.reindex(columns=new_order)


,data_year,county_name,AGENCY_NAME,agency_id,Crimes Against Persons,Crimes Against Society,Crimes Against Property,All Other Offenses (Except Traffic),Suspicion,latitude,longitude,ori_number,agency_type_name
5,2017,ALAMEDA,Alameda Police Department,938,236,457,231,273,0,37.766888,-122.242730,CA0010100,City
6,2018,ALAMEDA,Alameda Police Department,938,257,445,211,250,0,37.766888,-122.242730,CA0010100,City
7,2019,ALAMEDA,Alameda Police Department,938,300,445,223,293,0,37.766888,-122.242730,CA0010100,City
8,2020,ALAMEDA,Alameda Police Department,938,266,217,178,182,0,37.766888,-122.242730,CA0010100,City
9,2021,ALAMEDA,Alameda Police Department,938,27,55,30,28,0,37.766888,-122.242730,CA0010100,City
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2415,2018,YOLO,UC Davis Police Department,1813,17,27,13,20,0,38.679595,-121.902444,CA0579700,University or College
2416,2019,YOLO,UC Davis Police Department,1813,9,34,25,16,0,38.679595,-121.902444,CA0579700,University or College
2417,2020,YOLO,UC Davis Police Department,1813,8,13,7,8,0,38.679595,-121.902444,CA0579700,University or College
2418,2021,YOLO,UC Davis Police Department,1813,1,1,0,3,0,38.679595,-121.902444,CA0579700,University or College


In [91]:
#creates a dataframe grouped by county for the consolidated categories
county_level_data_consolidated = consolidated_crime_df.groupby(['county_name','data_year']).agg({
    'Crimes Against Persons': 'sum',
             'Crimes Against Society': 'sum',
             'Crimes Against Property': 'sum',
})


In [112]:
# Calculate the count of unique agencies for each county and year
unique_agency_count_consolidated = agencies_with_all_years.groupby(['county_name', 'data_year'])['AGENCY_NAME'].nunique().reset_index()

# Merge the unique agency count with county_level_data
county_level_data_consolidated = county_level_data_consolidated.merge(unique_agency_count_consolidated, on=['county_name', 'data_year'], how='left')
county_level_data_consolidated.rename(columns={'AGENCY_NAME': 'agency_count'}, inplace=True)


,county_name,data_year,Crimes Against Persons,Crimes Against Society,Crimes Against Property,agency_count
0,ALAMEDA,2017,517,1157,972,4
1,ALAMEDA,2018,556,1182,889,4
2,ALAMEDA,2019,600,1063,781,4
3,ALAMEDA,2020,458,634,578,4
4,ALAMEDA,2021,105,183,278,4
...,...,...,...,...,...,...
253,YOLO,2018,472,1033,450,3
254,YOLO,2019,422,1059,417,3
255,YOLO,2020,419,644,379,3
256,YOLO,2021,86,143,89,3


In [113]:
#creates a csv with the dataframe
county_level_data_consolidated.to_csv('Output/county_level_crime_data_consolidated.csv')

In [92]:
#creates a dataframe grouped by agency for the consolidated categories
agency_level_data_consolidated = consolidated_crime_df.groupby(['AGENCY_NAME','data_year']).agg({
    'Crimes Against Persons': 'sum',
             'Crimes Against Society': 'sum',
             'Crimes Against Property': 'sum',
})

#creates a csv with the dataframe
agency_level_data_consolidated.to_csv('Output/agency_level_crime_data_consolidated.csv')

In [238]:
#reads in file with excel method
file_path = 'poverty_population_county.csv'

poverty_data = pd.read_csv(file_path)

In [239]:
#change case of the county column to fit with the existing data sets
poverty_data['COUNTY_NAME'] = poverty_data['COUNTY_NAME'].str.upper()

In [240]:
#change the case of the county name column to match our existing dataframe
poverty_data = poverty_data.rename(columns={'COUNTY_NAME':'county_name'})

,county_name,POVERTY_RATE,POPULATION
0,ALAMEDA,12.0,"1,487,888"
1,ALPINE,13.7,"1,160"
2,AMADOR,10.5,"33,228"
3,BUTTE,20.6,"214,624"
4,CALAVERAS,10.4,"44,740"


In [241]:
#removes a trailing space from the county name in the dataframe being merged
poverty_data["county_name"] = poverty_data["county_name"].apply(lambda x: x.strip())

In [247]:
del consolidated_crime_full_counties["level_0"]
del consolidated_crime_full_counties["index"]


,county_name,POVERTY_RATE,POPULATION
0,ALAMEDA,12.0,"1,487,888"
1,ALPINE,13.7,"1,160"
2,AMADOR,10.5,"33,228"
3,BUTTE,20.6,"214,624"
4,CALAVERAS,10.4,"44,740"
5,COLUSA,15.2,"21,041"
6,CONTRA COSTA,10.2,"1,042,528"
7,DEL NORTE,21.5,"24,974"
8,EL DORADO,8.1,"179,280"
9,FRESNO,24.8,"913,669"


In [248]:
#merging the consolidated dataframe

population_consol_df = pd.merge(consolidated_crime_full_counties, poverty_data, on='county_name', how = 'inner')

,county_name,data_year,Crimes Against Persons,Crimes Against Society,Crimes Against Property,POVERTY_RATE,POPULATION
0,ALAMEDA,2017,7284,12156,8169,12.0,"1,487,888"
1,ALAMEDA,2018,7242,11225,7670,12.0,"1,487,888"
2,ALAMEDA,2019,7188,10867,7321,12.0,"1,487,888"
3,ALAMEDA,2020,6301,6507,6477,12.0,"1,487,888"
4,ALAMEDA,2021,105,183,278,12.0,"1,487,888"
...,...,...,...,...,...,...,...
328,YUBA,2017,618,1093,506,20.6,"70,740"
329,YUBA,2018,530,1127,412,20.6,"70,740"
330,YUBA,2019,540,1252,419,20.6,"70,740"
331,YUBA,2020,438,961,308,20.6,"70,740"


In [251]:
#creates csv from dataframe
population_consol_df.to_csv('Output/county_level_consolidated_with_census_info.csv')

In [230]:
#merge the data

population_disagg_crime_df = pd.merge(county_full, poverty_data, on='county_name', how = 'inner')

,county_name,data_year,Aggravated Assault,All Other Offenses (Except Traffic),Arson,Burglary,Curfew and Loitering Law Violations,Disorderly Conduct,Driving Under the Influence,Drug Abuse Violations - Grand Total,...,Simple Assault,"Stolen Property: Buying, Receiving, Possessing",Suspicion,Vagrancy,Vandalism,"Weapons: Carrying, Possessing, Etc.","Sex Offenses (Except Rape, and Prostitution and Commercialized Vice)",agency_count,POVERTY_RATE,POPULATION
0,ALAMEDA,2017,2625,8239,52,1082,0,156,1905,7556,...,4000,629,0,77,922,1024,457,18,12.0,"1,487,888"
1,ALAMEDA,2018,2725,7733,71,1014,0,157,1855,7153,...,4013,574,0,77,853,946,360,18,12.0,"1,487,888"
2,ALAMEDA,2019,2627,6919,41,1087,0,170,2031,6910,...,4165,565,0,58,765,908,278,18,12.0,"1,487,888"
3,ALAMEDA,2020,2524,4357,65,990,0,89,1253,4344,...,3408,690,0,45,673,939,234,18,12.0,"1,487,888"
4,ALAMEDA,2021,21,136,1,12,0,33,100,48,...,78,32,0,2,18,29,3,4,12.0,"1,487,888"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,YUBA,2017,388,757,12,94,0,1,252,372,...,157,39,0,5,45,91,38,3,20.6,"70,740"
329,YUBA,2018,336,790,9,89,0,5,179,445,...,151,20,0,2,34,82,21,3,20.6,"70,740"
330,YUBA,2019,323,830,4,82,0,3,197,552,...,171,26,0,0,34,104,27,3,20.6,"70,740"
331,YUBA,2020,276,503,6,62,0,0,117,589,...,118,28,0,3,48,61,37,3,20.6,"70,740"


In [252]:
#creates csv from dataframe
population_disagg_crime_df.to_csv('Output/county_level_non_consolidated.csv')